In [1]:
!pip install torchsde soundfile diffusers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.1 MB/s eta 0:00:00


In [2]:
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from IPython.display import Audio


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [3]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN_2026')

login(token=hf_token, add_to_git_credential=True)

print("HF login successful")


HF login successful


In [4]:
pipe = StableAudioPipeline.from_pretrained(
    "stabilityai/stable-audio-open-1.0",
    torch_dtype=torch.float16
)

pipe = pipe.to("cuda")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [5]:
prompts = [
    "strange jungle creature call, eerie organic tones",
    "alien rainforest animal sound, wet textures",
    "mysterious glowing beast vocalization",
    "dark lo-fi jungle ambience, surreal ecosystem"
]

negative_prompt = "distortion, noise artifacts, low quality"


In [7]:
import random

generator = torch.Generator("cuda")

for i, prompt in enumerate(prompts):

    duration = random.uniform(2.0, 20.0)  # 🔥 Random length (2s → 20s)

    audio = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,

        num_inference_steps=50,
        audio_end_in_s=duration,   # ✅ Randomized duration
        num_waveforms_per_prompt=1,

        generator=generator,
    ).audios

    output = audio[0].T.float().cpu().numpy()

    sf.write(f"sound_{i}.wav", output, pipe.vae.sampling_rate)

    print(f"Generated sound_{i}.wav ({duration:.1f}s)")


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:608: UserWarning: Should have tb<=t1 but got tb=500.00006103515625 and t1=500.0.
  warnings.warn(f"Should have {tb_name}<=t1 but got {tb_name}={tb} and t1={self._end}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.29999998211860657 and t0=0.3.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.0 and t0=0.3.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:602: UserWarning: Should have tb>=t0 but got tb=0.29999998211860657 and t0=0.3.
  warnings.warn(f"Should have {tb_name}>=t0 but got {tb_name}={tb} and t0={self._start}.")


Generated sound_0.wav (10.3s)


  0%|          | 0/50 [00:00<?, ?it/s]

Generated sound_1.wav (2.2s)


  0%|          | 0/50 [00:00<?, ?it/s]

Generated sound_2.wav (7.6s)


  0%|          | 0/50 [00:00<?, ?it/s]

Generated sound_3.wav (13.3s)


In [9]:
Audio("sound_0.wav")


In [8]:
!zip -r jungle_sounds.zip sound_*


  adding: sound_0.wav (deflated 18%)
  adding: sound_1.wav (deflated 38%)
  adding: sound_2.wav (deflated 22%)
  adding: sound_3.wav (deflated 13%)
